<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


- Install the additional package requirements for this bonus notebook by uncommenting and running the following cell:

In [1]:
# pip install -r requirements-extra.txt

# Comparing Various Byte Pair Encoding (BPE) Implementations

<br>
&nbsp;

## Using BPE from `tiktoken`

In [2]:
from importlib.metadata import version

print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.7.0


In [3]:
import tiktoken

tik_tokenizer = tiktoken.get_encoding("gpt2")

text = "Hello, world. Is this-- a test?"

In [4]:
integers = tik_tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]


In [5]:
strings = tik_tokenizer.decode(integers)

print(strings)

Hello, world. Is this-- a test?


In [6]:
print(tik_tokenizer.n_vocab)

50257


<br>
&nbsp;

## Using the original BPE implementation used in GPT-2

In [7]:
from bpe_openai_gpt2 import get_encoder, download_vocab

In [8]:
download_vocab()

Fetching encoder.json: 1.04Mit [00:00, 3.47Mit/s]                                                   
Fetching vocab.bpe: 457kit [00:00, 2.07Mit/s]                                                       


In [9]:
orig_tokenizer = get_encoder(model_name="gpt2_model", models_dir=".")

In [10]:
integers = orig_tokenizer.encode(text)

print(integers)

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]


In [11]:
strings = orig_tokenizer.decode(integers)

print(strings)

Hello, world. Is this-- a test?


<br>
&nbsp;

## Using the BPE via Hugging Face transformers

In [12]:
import transformers

transformers.__version__

'4.48.0'

In [13]:
from transformers import GPT2Tokenizer

hf_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [14]:
hf_tokenizer(strings)["input_ids"]

[15496, 11, 995, 13, 1148, 428, 438, 257, 1332, 30]

<br>
&nbsp;

## Using my own from-scratch BPE tokenizer

In [15]:
import os
import sys
import io
import nbformat
import types

def import_from_notebook():
    def import_definitions_from_notebook(fullname, names):
        current_dir = os.getcwd()
        path = os.path.join(current_dir, "..", "05_bpe-from-scratch", fullname + ".ipynb")
        path = os.path.normpath(path)

        # Load the notebook
        if not os.path.exists(path):
            raise FileNotFoundError(f"Notebook file not found at: {path}")

        with io.open(path, "r", encoding="utf-8") as f:
            nb = nbformat.read(f, as_version=4)

        # Create a module to store the imported functions and classes
        mod = types.ModuleType(fullname)
        sys.modules[fullname] = mod

        # Go through the notebook cells and only execute function or class definitions
        for cell in nb.cells:
            if cell.cell_type == "code":
                cell_code = cell.source
                for name in names:
                    # Check for function or class definitions
                    if f"def {name}" in cell_code or f"class {name}" in cell_code:
                        exec(cell_code, mod.__dict__)
        return mod

    fullname = "bpe-from-scratch"
    names = ["BPETokenizerSimple"]

    return import_definitions_from_notebook(fullname, names)

In [16]:
imported_module = import_from_notebook()
BPETokenizerSimple = getattr(imported_module, "BPETokenizerSimple", None)

tokenizer_gpt2 = BPETokenizerSimple()
tokenizer_gpt2.load_vocab_and_merges_from_openai(
    vocab_path=os.path.join("gpt2_model", "encoder.json"),
    bpe_merges_path=os.path.join("gpt2_model", "vocab.bpe")
)

In [17]:
integers = tokenizer_gpt2.encode(text)

print(integers)

[1544, 18798, 11, 995, 13, 1148, 256, 5303, 82, 438, 257, 1332, 30]


<br>
&nbsp;

## A quick performance benchmark

In [18]:
with open('../01_main-chapter-code/the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()

### Original OpenAI GPT-2 tokenizer

In [19]:
%timeit orig_tokenizer.encode(raw_text)

3.44 ms ± 54 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Tiktoken OpenAI GPT-2 tokenizer

In [20]:
%timeit tik_tokenizer.encode(raw_text)

1.08 ms ± 4.69 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Hugging Face OpenAI GPT-2 tokenizer

In [21]:
%timeit hf_tokenizer(raw_text)["input_ids"]

Token indices sequence length is longer than the specified maximum sequence length for this model (5145 > 1024). Running this sequence through the model will result in indexing errors


10.3 ms ± 180 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%timeit hf_tokenizer(raw_text, max_length=5145, truncation=True)["input_ids"]

10.2 ms ± 72.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### My own GPT-2 tokenizer (for educational purposes)

In [23]:
%timeit tokenizer_gpt2.encode(raw_text)

1.74 ms ± 48.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
